In [1]:
from matching import load_seqs, paired_fasta_to_labeled_array, split_labeled_seqs
import numpy as np
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import SeqIO

In [2]:
good_seqs, good_ids = load_seqs("./CODE/DataAttentionDCA/HK-RR/HK-RR_174_train.fasta", return_ID=True)

In [3]:

seqsA_label, seqsB_label = paired_fasta_to_labeled_array(
    "./CODE/DataAttentionDCA/HK-RR/HK-RR_174_train.fasta", L_A=63
)



seqsA_label[:,1]=seqsA_label[:,1].astype(np.uint16) # Needed for numba
seqsB_label[:,1]=seqsB_label[:,1].astype(np.uint16)

seqsA_split, seqsB_split = split_labeled_seqs(seqsA_label, seqsB_label)

seqsB_split_perm = np.array(seqsB_split, dtype=object)
nb_species = len(seqsA_split)
fixed_points=0
permutations=[]
for species in range(nb_species):

    seqsA_tmp = seqsA_split[species]
    seqsB_tmp = seqsB_split[species]

    permutation = np.random.permutation(seqsB_tmp.shape[0])
    permutations.append(permutation)

    seqsB_split_perm[species] = seqsB_tmp[permutation]


    fixed_points += np.sum(permutation == np.arange(len(permutation)))

print(fixed_points)

1942


In [86]:
with open("./CODE/DataAttentionDCA/HK-RR/HK-RR_174_train_scambled_permutations.txt", "w") as f:
	for permutation in permutations:
		np.savetxt(f, permutation[None], fmt='%d')

In [87]:
# put sequences back together:

seqsA= np.concatenate(seqsA_split)
seqsA = seqsA[np.lexsort((seqsA[:, 1], seqsA[:, 0]))]

seqsB = np.concatenate(seqsB_split_perm)

In [88]:
ids_A = good_ids[seqsA[:, 1].astype(int)]
ids_B = good_ids[seqsB[:, 1].astype(int)]

ids_A=[id.split("|")[0]for id in ids_A]
ids_B=[id.split("//")[1].split("|")[0] +"|"+id.split("|")[1] for id in ids_B]

In [89]:
all_seqs_new = np.concatenate((seqsA[:, 2:], seqsB[:, 2:]), axis=1)
all_seqs_new

array([['-', '-', '-', ..., 'K', 'V', '-'],
       ['-', 'L', 'S', ..., 'R', 'V', '-'],
       ['-', '-', 'N', ..., 'R', 'I', 'K'],
       ...,
       ['-', '-', 'K', ..., 'R', '-', '-'],
       ['-', 'L', 'N', ..., 'R', '-', '-'],
       ['-', 'Q', 'N', ..., 'R', '-', '-']], dtype=object)

In [90]:
out_file    = open("./CODE/DataAttentionDCA/HK-RR/HK-RR_174_train_scambled.fasta", "w")
for i in range(all_seqs_new.shape[0]):
    record = SeqRecord(
    Seq(''.join(all_seqs_new[i])),
    id=ids_A[i]+"/"+ids_B[i],
    description="")
    SeqIO.write(record, out_file, "fasta")